In [1]:
import sys
sys.path.insert(0, '/root/mounted/projects/qpair/')

In [2]:
from config import *

In [3]:
# 加载模块
import pandas as pd
import pickle
import os
import numpy as np
from add_vectors_to_qpairs import add_wv_to_qpairs
from keras.preprocessing.sequence import pad_sequences

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
# KN_PAIRS_SAMPLED_PATH = '/root/mounted/datasets/data_0521/intermediate/'
# KN_PAIRS_SAMPLED_PATH = '/root/mounted/datasets/data_0523/publish/'
KN_PAIRS_SAMPLED_PATH = '/root/mounted/datasets/publish_0606_tr50_v3.bak/'
# KN_PAIRS_SAMPLED_PATH = '/root/mounted/datasets/publish_0606_tr50_test/'
# KN_PAIRS_SAMPLED_PATH = '/root/mounted/datasets/publish_mingwen_test/'
KN_PAIRS_SAMPLED_TRAIN_CSV = os.path.join(KN_PAIRS_SAMPLED_PATH, 'frontend/train.csv')
KN_PAIRS_SAMPLED_TEST_CSV = os.path.join(KN_PAIRS_SAMPLED_PATH, 'frontend/test.csv')
QUESTION_CSV = os.path.join(KN_PAIRS_SAMPLED_PATH, 'frontend/question.csv')
INTERMEDIATE_DATA_PATH = os.path.join(KN_PAIRS_SAMPLED_PATH, 'intermediate')
WEM_PKL = os.path.join(KN_PAIRS_SAMPLED_PATH, 'intermediate/WEM.pkl') # 覆盖config
CEM_PKL = os.path.join(KN_PAIRS_SAMPLED_PATH, 'intermediate/CEM.pkl') # 覆盖config

if not os.path.exists(INTERMEDIATE_DATA_PATH):
    os.mkdir(INTERMEDIATE_DATA_PATH)

WEM_PKL, CEM_PKL

('/root/mounted/datasets/publish_0606_tr50_v3.bak/intermediate/WEM.pkl',
 '/root/mounted/datasets/publish_0606_tr50_v3.bak/intermediate/CEM.pkl')

In [5]:
kn_pairs_sampled_train = pd.read_csv(KN_PAIRS_SAMPLED_TRAIN_CSV)
kn_pairs_sampled_train

,label,q1,q2
0,1,Q397345,Q538594
1,0,Q193805,Q699273
2,0,Q085471,Q676160
3,0,Q189314,Q438123
4,0,Q267714,Q290126
5,1,Q369715,Q658908
6,1,Q537991,Q268444
7,0,Q255847,Q179358
8,1,Q639518,Q053248
9,0,Q521550,Q435529


In [6]:
# kn_pairs_sampled_train.columns = ['label', 'qid1', 'qid2', 'q1_q2_intersect']
kn_pairs_sampled_train.columns = ['label', 'qid1', 'qid2']
kn_pairs_sampled_train

,label,qid1,qid2
0,1,Q397345,Q538594
1,0,Q193805,Q699273
2,0,Q085471,Q676160
3,0,Q189314,Q438123
4,0,Q267714,Q290126
5,1,Q369715,Q658908
6,1,Q537991,Q268444
7,0,Q255847,Q179358
8,1,Q639518,Q053248
9,0,Q521550,Q435529


In [7]:
kn_pairs_sampled_test = pd.read_csv(KN_PAIRS_SAMPLED_TEST_CSV)
kn_pairs_sampled_test

,q1,q2
0,Q017571,Q006012
1,Q728241,Q542572
2,Q166997,Q118270
3,Q422420,Q514386
4,Q354329,Q032485
5,Q438856,Q050667
6,Q635647,Q217357
7,Q466258,Q543640
8,Q244004,Q289633
9,Q332566,Q058858


In [8]:
# kn_pairs_sampled_test.columns = ['label', 'qid1', 'qid2', 'q1_q2_intersect']
kn_pairs_sampled_test.columns = [ 'qid1', 'qid2']
kn_pairs_sampled_test

,qid1,qid2
0,Q017571,Q006012
1,Q728241,Q542572
2,Q166997,Q118270
3,Q422420,Q514386
4,Q354329,Q032485
5,Q438856,Q050667
6,Q635647,Q217357
7,Q466258,Q543640
8,Q244004,Q289633
9,Q332566,Q058858


In [9]:
# with open(QUESTION_PKL, 'rb') as f:
#     qs = pickle.load(f)

# qs
qs = pd.read_csv(QUESTION_CSV)
qs

,qid,words,chars
0,Q000000,W05733 W05284 W09158 W14968 W07863,L1128 L1861 L2218 L1796 L1055 L0847 L2927
1,Q000001,W17378 W17534 W03249 W01490 W18802,L2214 L1980 L0156 L1554 L2218 L1861 L3019 L010...
2,Q000002,W17378 W08158 W20171 W11246 W14759,L2214 L2350 L2568 L1969 L2168 L0694 L3012 L256...
3,Q000003,W11385 W14103 W02556 W13157 W09749,L0762 L2172 L1971 L2034 L2039 L2271 L1346 L223...
4,Q000004,W17508 W18238 W02952 W18103,L0018 L2321 L1346 L3019 L0104 L0902 L0354 L0362
5,Q000005,W17378 W12908 W19355 W08410 W11922 W14496 W20487,L2214 L1331 L0016 L1132 L2237 L1900 L0578 L195...
6,Q000006,W04346 W17378 W06112 W05733 W18238 W05284 W118...,L2323 L1526 L2214 L0358 L1849 L1128 L1861 L301...
7,Q000007,W16696 W02556 W16564 W03914 W18103,L0366 L2028 L2034 L2039 L0605 L1824 L2292 L015...
8,Q000008,W17378 W11385 W18238 W13157 W12244 W16319 W18448,L2214 L0762 L2172 L3019 L0104 L2271 L1346 L014...
9,Q000009,W14113 W16319 W18399 W03472 W11399 W07863 W18103,L0947 L2323 L2582 L0143 L2600 L2272 L1872 L123...


In [10]:
with open(WEM_PKL, 'rb') as f:
    wmap = pickle.load(f)

w2i = wmap['word2index']
w2i

{'W13158': 2646,
 'W06050': 7934,
 'W06035': 16115,
 'W15596': 12532,
 'W19634': 4331,
 'W00304': 15846,
 'W03166': 5514,
 'W09606': 16301,
 'W15117': 9425,
 'W20374': 5754,
 'W10601': 5454,
 'W04568': 2796,
 'W05666': 15760,
 'W17007': 10175,
 'W02933': 3916,
 'W12671': 3583,
 'W07201': 12514,
 'W12734': 12076,
 'W11002': 15425,
 'W04183': 12651,
 'W10629': 4418,
 'W20560': 18655,
 'W03145': 5923,
 'W15384': 17734,
 'W08262': 5748,
 'W07321': 16916,
 'W05016': 6322,
 'W20471': 8593,
 'W06303': 3545,
 'W02882': 19325,
 'W00209': 8375,
 'W11675': 823,
 'W10653': 2806,
 'W02472': 12336,
 'W17412': 17558,
 'W19657': 2750,
 'W07360': 17492,
 'W10975': 13533,
 'W13046': 17326,
 'W07422': 9497,
 'W06134': 1020,
 'W00995': 6471,
 'W07155': 10823,
 'W13087': 9365,
 'W09091': 19917,
 'W12021': 543,
 'W19161': 20706,
 'W02052': 4562,
 'W11329': 1237,
 'W20210': 3729,
 'W08608': 18770,
 'W00301': 20183,
 'W11011': 19769,
 'W11271': 1204,
 'W00664': 17481,
 'W09927': 150,
 'W16597': 2053,
 'W14745

In [11]:
with open(CEM_PKL, 'rb') as f:
    cmap = pickle.load(f)

c2i = cmap['char2index']
c2i

{'L2711': 2842,
 'L1728': 3045,
 'L0578': 560,
 'L0611': 2892,
 'L2761': 2977,
 'L1287': 2,
 'L1157': 991,
 'L2432': 2750,
 'L0604': 2410,
 'L1843': 1420,
 'L2646': 2528,
 'L0909': 894,
 'L2536': 1432,
 'L0029': 1925,
 'L1900': 1911,
 'L0542': 1345,
 'L1168': 930,
 'L0418': 287,
 'L1693': 729,
 'L1983': 2464,
 'L2521': 133,
 'L0180': 66,
 'L1147': 442,
 'L1384': 1435,
 'L2149': 656,
 'L0362': 786,
 'L0143': 382,
 'L0558': 45,
 'L2901': 801,
 'L2125': 2930,
 'L0495': 2689,
 'L2900': 121,
 'L0045': 2001,
 'L2817': 1815,
 'L2206': 1545,
 'L2987': 207,
 'L1247': 2869,
 'L2101': 1036,
 'L0944': 1801,
 'L1678': 2390,
 'L2202': 1869,
 'L2398': 84,
 'L1992': 541,
 'L2726': 60,
 'L2473': 2507,
 'L0663': 2899,
 'L0035': 1271,
 'L2728': 971,
 'L0801': 688,
 'L3005': 162,
 'L0358': 1413,
 'L0420': 1768,
 'L1304': 1108,
 'L1999': 34,
 'L0772': 1531,
 'L1634': 2052,
 'L1523': 1767,
 'L1687': 2951,
 'L1122': 51,
 'L1411': 1136,
 'L1633': 1916,
 'L2181': 2759,
 'L0576': 1127,
 'L0677': 1397,
 'L1558':

In [12]:
# WEM_TXT = os.path.join(KN_PAIRS_SAMPLED_PATH, 'frontend/word_embed.txt')
# CEM_TXT = os.path.join(KN_PAIRS_SAMPLED_PATH, 'frontend/char_embed.txt')

# def get_em(txt):
#     res = {}
#     with open(txt, 'r') as f:
#         for line in f:
#             key_em = line.split()
#             res[key_em[0]] = np.array(key_em[1:], dtype=np.float32)
    
#     return res

# w2i = get_em(WEM_TXT)
# c2i = get_em(CEM_TXT)

In [13]:
# w2i

In [14]:
# c2i

In [15]:
kn_pairs_sampled_wids_cids_train = add_wv_to_qpairs(kn_pairs_sampled_train, qs, w2i, c2i, kn_pairs_sampled_train.shape[0])
kn_pairs_sampled_wids_cids_train

254387it [06:40, 635.06it/s]                            


,label,qid1,qid2,words1,words2,chars1,chars2
0,1,Q397345,Q538594,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,0,Q193805,Q699273,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,0,Q085471,Q676160,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,0,Q189314,Q438123,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,0,Q267714,Q290126,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
5,1,Q369715,Q658908,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
6,1,Q537991,Q268444,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
7,0,Q255847,Q179358,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
8,1,Q639518,Q053248,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
9,0,Q521550,Q435529,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [16]:
kn_pairs_sampled_wids_cids_test = add_wv_to_qpairs(kn_pairs_sampled_test, qs, w2i, c2i, kn_pairs_sampled_test.shape[0])
kn_pairs_sampled_wids_cids_test.head()

172957it [04:31, 636.07it/s]                            


,qid1,qid2,words1,words2,chars1,chars2
0,Q017571,Q006012,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,Q728241,Q542572,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,Q166997,Q118270,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,Q422420,Q514386,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,Q354329,Q032485,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [17]:
kn_pairs_sampled_test.head()

,qid1,qid2
0,Q017571,Q006012
1,Q728241,Q542572
2,Q166997,Q118270
3,Q422420,Q514386
4,Q354329,Q032485


In [18]:
# 保存
KN_PAIRS_SAMPLED_WIDS_CIDS_TRAIN_PKL = os.path.join(KN_PAIRS_SAMPLED_PATH, 'intermediate/train.pkl')
KN_PAIRS_SAMPLED_WIDS_CIDS_TEST_PKL = os.path.join(KN_PAIRS_SAMPLED_PATH, 'intermediate/test.pkl')
with open(KN_PAIRS_SAMPLED_WIDS_CIDS_TRAIN_PKL, 'wb') as f:
    pickle.dump(kn_pairs_sampled_wids_cids_train, f)

with open(KN_PAIRS_SAMPLED_WIDS_CIDS_TEST_PKL, 'wb') as f:
    pickle.dump(kn_pairs_sampled_wids_cids_test, f)
